In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

from astropy.io import fits
from astropy.table import Table
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
import astropy.units as u

import sys
import os
import glob
from tqdm import tqdm

Exception ignored in: <module 'collections.abc' from 'C:\\Users\\oryan\\AppData\\Local\\Continuum\\anaconda3\\lib\\collections\\abc.py'>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
KeyboardInterrupt
C:\Users\oryan\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: Cython module failed to patch module with custom type
  return f(*args, **kwds)

KeyboardInterrupt



In [ ]:
folder = 'C:/Users/oryan/Documents/mergers-in-cosmos'
data_folder = f'{folder}/data'
results = f'{folder}/results'

drive_folder = 'E:/cosmos-data'

## Importing Data

In [ ]:
with fits.open(f'{drive_folder}/COSMOS2020_CLASSIC_R1_v2.1_p3.fits.gz') as hdul:
    header = hdul[1].header
    data = hdul[1].data

In [4]:
df = pd.read_csv(f'{data_folder}/catalogue-matched-cosmos-2020.csv', index_col = 0)

In [5]:
df.head()

,SourceID,ID_1,ALPHA_J2000_1,DELTA_J2000_1,X_IMAGE_1,Y_IMAGE_1,ERRX2_IMAGE_1,ERRY2_IMAGE_1,ERRXY_IMAGE_1,FLUX_RADIUS_1,...,ez_ssfr_p025_2,ez_ssfr_p160_2,ez_ssfr_p500_2,ez_ssfr_p840_2,ez_ssfr_p975_2,ez_Av_p025_2,ez_Av_p160_2,ez_Av_p500_2,ez_Av_p840_2,ez_Av_p975_2
0,4000705532984,857121.0,150.673667,2.226291,9348.870117,22451.160156,0.000002,0.000004,1.265998e-08,8.876858,...,-8.385987,-8.323476,-8.259419,-8.171948,-8.081048,0.595371,0.783795,0.924471,1.022045,1.076083
1,4000705533312,873195.0,150.668102,2.242849,9482.499023,22848.505859,0.000007,0.000002,1.829277e-06,5.542504,...,-8.962106,-8.711401,-8.478580,-8.303174,-8.135300,0.082679,0.231435,0.436111,0.622873,0.829844
2,4000705533383,861738.0,150.645118,2.237538,10033.689453,22720.843750,0.000039,0.000078,1.332813e-05,5.169795,...,-9.779914,-9.521317,-9.042374,-8.946216,-8.919963,0.570974,0.686736,0.964232,1.396826,1.587413
3,4000705539435,1280765.0,149.702469,2.636086,32637.894531,32285.564453,0.000561,0.000900,1.408110e-05,4.764572,...,-10.821019,-10.378546,-10.191748,-10.048404,-9.860973,0.251250,0.657133,1.055286,1.348915,1.713512
4,4000705539529,1284864.0,149.686223,2.637412,33027.406250,32317.517578,0.000031,0.000030,1.244353e-06,6.412690,...,-8.389942,-8.324630,-8.253101,-8.180449,-8.090549,0.360573,0.404528,0.450688,0.494164,0.550538


## Calculating Environment Parameter

The equation we will use to calculate the enviroment parameter is the following:

Sigma_N = N / (pi * d**2)

This is then logged and averaged between the 4th and 5th neighbour. Once we have this, begin following the Baldry et al (2006) further. I'm not sure what units d is meant to be in and how it gets normalised to be unitless.

Sigma should be in units of Mpc**-2!

In [17]:
df_red = (
    df[['SourceID', 'ID_1', 'ALPHA_J2000_1', 'DELTA_J2000_1']]
)

In [18]:
coord_dict = df_red[:50].set_index('SourceID').to_dict(orient = 'index')

In [28]:
def calc_sep(ra1, dec1, ra2, dec2, z_1, z_2, cosmo):
    
    d1 = cosmo.comoving_distance(z_1).to(u.kpc)
    d2 = cosmo.comoving_distance(z_1).to(u.kpc)
    
    c1 = SkyCoord(ra = ra1 * u.deg, dec = dec1 * u.deg, frame = 'fk5')
    c2 = SkyCoord(ra = ra2 * u.deg, dec = dec2 * u.deg, frame = 'fk5')
    
    ang_sep = c1.separation(c2).to(u.arcmin)
    conversion = cosmo.kpc_proper_per_arcmin(z_1)
    
    c1 = SkyCoord(ra = ra1 * u.deg, dec = dec1 * u.deg, distance = d1, frame = 'fk5')
    c2 = SkyCoord(ra = ra2 * u.deg, dec = dec2 * u.deg, distance = d2, frame = 'fk5')
    
    # sep = c1.separation(c2)
    
    proj_sep = ang_sep * conversion
    
    return float(proj_sep.to(u.Mpc) / u.Mpc)

In [20]:
def get_n_neighbours(gal_id, ra, dec):
    
    clear_output(wait = True)
    end_index = 0
    ang = 0.1
    while end_index < 5:
        record = data[(data['ALPHA_J2000'] < (ra + ang)) & (data['ALPHA_J2000'] > (ra - ang)) & (data['DELTA_J2000'] < (dec + ang)) & (data['DELTA_J2000'] > (dec - ang))]

        table = Table(record)

        prim_galaxy_record = record[record['ID'] == gal_id]
        prim_z = prim_galaxy_record['ez_z_phot'][0]

        df = table.to_pandas()[['ID','ALPHA_J2000', 'DELTA_J2000', 'ez_z_phot', 'lp_type']]
        df = df.query('lp_type == 0').drop(columns = 'lp_type').dropna()
        df = df.query('ID != @gal_id')

        if len(df) < 0.5:
            return 'null'

        df_z_diff = (
            df
            .assign(z_diff = df.ez_z_phot.apply(lambda x: abs(x - prim_z)))
        )

        df_z_cut = df_z_diff.query('z_diff < 0.005')
        if len(df_z_cut) < 1:
            return {'IDs': [], 'separations': [], 'N_1' : None, 'N_2' : None, 'N_3' : None, 'N_4': None, 'N_5' : None}

        df_sep = (
            df_z_cut
            .assign(separation = df_z_cut.apply(lambda row: calc_sep(ra, dec, row.ALPHA_J2000, row.DELTA_J2000, prim_z, row.ez_z_phot, cosmo), axis = 1))
        )

        df_sorted = df_sep.sort_values(by = 'separation', ascending = True)

        end_index = 5
        if len(df_sorted) < end_index:
            print('Expanding Search Range...')
            end_index = len(df_sorted)
            ang += 0.05

    df_nearest = df_sorted[:end_index]

    nearest = {'IDs': [], 'separations': [], 'N_1' : None, 'N_2' : None, 'N_3' : None, 'N_4': None, 'N_5' : None}
    for i in range(end_index):
        nearest['IDs'].append(df_nearest.ID.iloc[i])
        nearest['separations'].append(df_nearest.separation.iloc[i])
        nearest[f'N_{i+1}'] = i+1 / (np.pi * (df_nearest.separation.iloc[i])**2)
        
    return nearest

In [21]:
global cosmo
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)

## Below will take about 2 Hours.
Run in the morning!

In [22]:
# df_done = pd.read_csv(f'{results}/nearest-neighbours-corr-3d.csv', index_col = 0)

In [23]:
# done_list = list(df_done.SourceID)
done_list = []

In [24]:
global data

In [29]:
results_dict = {}
counter = 0
mult = 1
for sourceid, values in tqdm(coord_dict.items()):
    if sourceid in done_list:
        continue
    results_dict[sourceid] = get_n_neighbours(values['ID_1'], values['ALPHA_J2000_1'], values['DELTA_J2000_1'])
    counter += 1
    
    if counter / 50 == 10:
        df_Ns = pd.DataFrame.from_dict(results_dict, orient = 'index').reset_index()
        df_Ns.to_csv(f'{results}/tmp-nearest-neighbours-corr-3d-{mult*counter}.csv')
        counter = 0
        mult += 1
        del df_Ns
        results_dict = {}

100%|██████████| 50/50 [01:42<00:00,  2.06s/it]


In [30]:
df_Ns = pd.DataFrame.from_dict(results_dict, orient = 'index').reset_index()

In [35]:
csv_files = glob.glob(f'{results}/tmp-nearest-neighbours-corr-3d-*.csv')
for i in csv_files:
    df_tmp = pd.read_csv(i, index_col = 0)
    df_Ns = pd.concat([df_Ns, df_tmp])

In [38]:
df_Ns.to_csv(f'{results}/nearest-neighbours-corr-3d.csv')

In [41]:
for i in csv_files:
    os.remove(i)